# World Marathon Majors - Winners

The world marathon majors consist of six major city marathons (https://en.wikipedia.org/wiki/World_Marathon_Majors):

List of all historic winners can be found via their individual wikipedia pages:

    - Tokyo (https://en.wikipedia.org/wiki/Tokyo_Marathon)
    - Boston (https://en.wikipedia.org/wiki/List_of_winners_of_the_Boston_Marathon)
    - London (https://en.wikipedia.org/wiki/List_of_winners_of_the_London_Marathon)
    - Berlin (https://en.wikipedia.org/wiki/Berlin_Marathon)
    - Chicago (https://en.wikipedia.org/wiki/List_of_winners_of_the_Chicago_Marathon)
    - New York (https://en.wikipedia.org/wiki/List_of_winners_of_the_New_York_City_Marathon)
    
Using the Wikipedia API, lets see if we can collect and compile a list of winners for each race on both the male and female runners and wheelchair athletes races and process to an easy to read form.

In [163]:
import pandas as pd
import wikipedia as wp
import re

### Find each page and make sure each exists

In [35]:
Tokyo_page=wp.page('Tokyo Marathon')
Boston_page=wp.page('List of winners of the Boston Marathon')
London_page=wp.page('List of winners of the London Marathon')
Berlin_page=wp.page('Berlin Marathon')
Chicago_page=wp.page('List of winners of the Chicago Marathon')
New_York_page=wp.page('List of winners of the New York City Marathon')

pages=[Tokyo_page,Boston_page,London_page,Berlin_page,Chicago_page,New_York_page]

for page in pages:
    print(page.title)

Tokyo Marathon
List of winners of the Boston Marathon
List of winners of the London Marathon
Berlin Marathon
List of winners of the Chicago Marathon
List of winners of the New York City Marathon


For each page, we'll use the pandas read_html() method to get all relevant tables.

We first need to identify which tables we need by manually inspecting the wikipedia pages for each race.

## The London Marathon

https://en.wikipedia.org/wiki/List_of_winners_of_the_London_Marathon

In [36]:
London_page.summary

"The London Marathon, one of the six World Marathon Majors, has been contested by men and women annually since 29 March 1981. Set over a largely flat course around the River Thames, the marathon is 26.2 miles (42.2 km) in length and generally regarded as a competitive and unpredictable event, and conducive to fast times.The inaugural marathon had 7,741 entrants, 6,255 of whom completed the race. The first Men's Elite Race was tied between American Dick Beardsley and Norwegian Inge Simonsen, who crossed the finish line holding hands in 2 hours, 11 minutes, 48 seconds. The first Women's Elite Race was won by Briton Joyce Smith in 2:29:57. In 1983, the first wheelchair races took place. Organized by the British Sports Association for the Disabled (BASD), 19 people competed and 17 finished. Gordon Perry of the United Kingdom won the Men's Wheelchair Race, coming in at 3:20:07, and Denise Smith, also of the UK, won the Women's Wheelchair Race in 4:29:03.Twenty athletes representing the Unit

By manually inspecting this page, we see that the results we're looking for are the first to fourth tables on the page.

In [148]:
html = wp.page("List of winners of the London Marathon").html()
male_elite_London = pd.read_html(html)[0]
female_elite_London = pd.read_html(html)[1]
male_wheel_London = pd.read_html(html)[2]
female_wheel_London = pd.read_html(html)[3]

In [149]:
male_elite_London.head()

,0,1,2,3,4
0,Year,Athlete,Nationality,Time(h:m:s),Notes
1,1981,Dick Beardsley (Tie),United States,2:11:48,Course record
2,Inge Simonsen (Tie),Norway,NaN,NaN,NaN
3,1982,Hugh Jones,United Kingdom,2:09:24,Course record
4,1983,Mike Gratton,United Kingdom,2:09:43,NaN


### Cleaning up

London is famous for having a tie in the men's race on its first edition in 1981. This is reflected in the first table taken from the wiki page so let's clean that up first. It's only one instance, so let's clean this manually.

In [150]:
male_elite_London.iloc[1][1]='Dick Beardsley and Inge Simonson (Tie)'
male_elite_London.iloc[1][2]='United States and Norway'
male_elite_London=male_elite_London.reindex(male_elite_London.index.drop(2)).reset_index(drop=True)
male_elite_London.head()

,0,1,2,3,4
0,Year,Athlete,Nationality,Time(h:m:s),Notes
1,1981,Dick Beardsley and Inge Simonson (Tie),United States and Norway,2:11:48,Course record
2,1982,Hugh Jones,United Kingdom,2:09:24,Course record
3,1983,Mike Gratton,United Kingdom,2:09:43,NaN
4,1984,Charlie Spedding,United Kingdom,2:09:57,NaN


Now that initial cleaning is done, let's standardise these tables somewhat making the year the index and change the column headings out of the first row. 

Let's also convert that time to datetime format.

In [152]:
def standardise_table(df):
    df.columns = df.iloc[0]
    df=df.reindex(df.index.drop(0))
    df=df.set_index('Year')
    df['Time(h:m:s)']=pd.to_datetime(df['Time(h:m:s)'],format='%H:%M:%S').dt.time
    return(df)

In [153]:
male_elite_London=standardise_table(male_elite_London)
female_elite_London=standardise_table(female_elite_London)
male_wheel_London=standardise_table(male_wheel_London)
female_wheel_London=standardise_table(female_wheel_London)

male_elite_London.head()

,Athlete,Nationality,Time(h:m:s),Notes
Year,,,,
1981,Dick Beardsley and Inge Simonson (Tie),United States and Norway,02:11:48,Course record
1982,Hugh Jones,United Kingdom,02:09:24,Course record
1983,Mike Gratton,United Kingdom,02:09:43,NaN
1984,Charlie Spedding,United Kingdom,02:09:57,NaN
1985,Steve Jones,United Kingdom,02:08:16,Course record


## Boston Marathon

https://en.wikipedia.org/wiki/List_of_winners_of_the_Boston_Marathon

In [154]:
Boston_page.summary

'The Boston Marathon is an annual marathon held in the Greater Boston area in Massachusetts. The event is held on Patriots Day, the third Monday of April. The Boston Marathon has been held annually since 1897 and is the oldest annual marathon in the world.'

In [308]:
html = wp.page("List of winners of the Boston Marathon").html()
male_elite_Boston = pd.read_html(html)[1]
female_elite_Boston = pd.read_html(html)[2]
male_wheel_Boston = pd.read_html(html)[3]
female_wheel_Boston = pd.read_html(html)[4]

In [309]:
male_wheel_Boston.head()

,0,1,2,3,4
0,Year,Athlete,Country/State,Time,Notes
1,1975,"Hall, RobertRobert Hall",United States United States (MA),2:58:00,NaN
2,1976,zzzNone,NaN,NaN,NaN
3,1977,"Hall, RobertRobert Hall",United States United States (MA),2:40:10,2nd victory
4,1978,"Murray, GeorgeGeorge Murray",United States United States (FL),2:26:57,NaN


There was no mens wheelchair race in 1976 so we'll remove that from this table.

In [310]:
male_wheel_Boston=male_wheel_Boston.reindex(male_wheel_Boston.index.drop(2)).reset_index(drop=True)
male_wheel_Boston.head()

,0,1,2,3,4
0,Year,Athlete,Country/State,Time,Notes
1,1975,"Hall, RobertRobert Hall",United States United States (MA),2:58:00,NaN
2,1977,"Hall, RobertRobert Hall",United States United States (MA),2:40:10,2nd victory
3,1978,"Murray, GeorgeGeorge Murray",United States United States (FL),2:26:57,NaN
4,1979,"Archer, KenKen Archer",United States United States (OH),2:38:59,NaN


For both the mens and women's wheelchair race, there are also time containing citations. We'll need to remove these as well so we can convert the time to datetime format. We can do that with a simple regex.

In [311]:
male_wheel_Boston.tail()

,0,1,2,3,4
39,2014,Ernst van Dyk,South Africa,1:20:36,10th victory
40,2015,Marcel Hug,Switzerland,1:29:53,NaN
41,2016,Marcel Hug,Switzerland,1:24:01,2nd victory
42,2017,Marcel Hug,Switzerland,1:18:03,3rd victory
43,2018,Marcel Hug,Switzerland,1:46:26[6],4th victory


In [312]:
def remove_citation(time):
    return(re.sub('\[.*?\]','',str(time)))

In [313]:
male_wheel_Boston[3]=male_wheel_Boston[3].apply(remove_citation)
female_wheel_Boston[3]=female_wheel_Boston[3].apply(remove_citation)
male_wheel_Boston.tail()

,0,1,2,3,4
39,2014,Ernst van Dyk,South Africa,1:20:36,10th victory
40,2015,Marcel Hug,Switzerland,1:29:53,NaN
41,2016,Marcel Hug,Switzerland,1:24:01,2nd victory
42,2017,Marcel Hug,Switzerland,1:18:03,3rd victory
43,2018,Marcel Hug,Switzerland,1:46:26,4th victory


### Standardise tables

As with the London Marathon tables, we can now make the year the index and change the column headings out of the first row. We'll also convert the time to datetime format.

In [314]:
def standardise_table(df):
    df.columns = df.iloc[0]
    df=df.reindex(df.index.drop(0))
    df=df.set_index('Year')
    df['Time']=pd.to_datetime(df['Time'],format='%H:%M:%S',errors='coerce').dt.time
    return(df)

In [315]:
male_elite_Boston=standardise_table(male_elite_Boston)
female_elite_Boston=standardise_table(female_elite_Boston)
male_wheel_Boston=standardise_table(male_wheel_Boston)
female_wheel_Boston=standardise_table(female_wheel_Boston)

male_elite_Boston.head()

,Athlete,Country/State or Province,Time,Notes
Year,,,,
1897,"McDermott, John J.John J. McDermott",United States United States(NY),02:55:10,NaN
1898,"MacDonald, Ronald J.Ronald J. MacDonald",Canada Canada (NS),02:42:00,NaN
1899,"Brignolia, LawrenceLawrence Brignolia",United States United States (MA),02:54:38,NaN
1900,"Caffery, JohnJohn ""Jack"" Caffery",Canada Canada (ON),02:39:44,NaN
1901,"Caffery, JohnJohn ""Jack"" Caffery",Canada Canada (ON),02:29:23,2nd victory


### Cleaning up

For the Boston results, we have an odd situation where many of the names in the tables were hyperlinked so we have repetitions where the cell of the table in the html contained text both in the span and in the anchor text of the url. For this we'll use regex.

In [316]:
male_elite_Boston['Athlete'].iloc[1]

'MacDonald, Ronald J.Ronald J. MacDonald'

In [317]:
def clean_names(athlete):
    #find all words in name starting with a capital letter followed by lower case
    regex='[A-Z][a-z]+'
    names=re.findall(regex,athlete)
    #join 'Mc' and 'Mac' to second part of the name and take care of double barrels
    exceptions=['Mc','Mac','De','Van','Cable']
    names_amended=[]
    for i,name in enumerate(names):
        if name in exceptions:
            names_amended.append(name+str(names[i+1]))
            names.remove(names[i+1])
        else:
            names_amended.append(name)
    #the full name should now be the last two words in the list
    names_amended
    name=' '.join(names_amended[len(names_amended)-2:])
    return(name)

In [318]:
male_elite_Boston['Athlete']=male_elite_Boston['Athlete'].apply(clean_names)
female_elite_Boston['Athlete']=female_elite_Boston['Athlete'].apply(clean_names)
male_wheel_Boston['Athlete']=male_wheel_Boston['Athlete'].apply(clean_names)
female_wheel_Boston['Athlete']=female_wheel_Boston['Athlete'].apply(clean_names)
male_elite_Boston.head()

,Athlete,Country/State or Province,Time,Notes
Year,,,,
1897,John McDermott,United States United States(NY),02:55:10,NaN
1898,Ronald MacDonald,Canada Canada (NS),02:42:00,NaN
1899,Lawrence Brignolia,United States United States (MA),02:54:38,NaN
1900,Jack Caffery,Canada Canada (ON),02:39:44,NaN
1901,Jack Caffery,Canada Canada (ON),02:29:23,2nd victory


The final cleaning step for the Boston results is to separate out the Country from the state for US and Canadian athletes.